In [2]:
from pathlib import Path
import sys

ROOT = Path.cwd().parent
sys.path.append(str(ROOT))

import os, glob, re
import pandas as pd
import numpy as np

from tqdm import tqdm
from pathlib import Path
from torch_geometric.data import Data

from lib.graph_builder import build_emergence_graphs_for_time_range
from lib.utils import load_parts

In [3]:
DATA_DIR = "../elliptic_dataset"
WALLETS_FEATURES = "wallets_features.csv"
WALLETS_CLASSES = "wallets_classes.csv"
EDGES_PREFIX = "AddrTxAddr_edgelist_part_"

In [4]:
nodes = pd.read_csv(os.path.join(DATA_DIR, WALLETS_FEATURES))
node_labels = pd.read_csv(os.path.join(DATA_DIR, WALLETS_CLASSES))
edges_with_edge_labels = load_parts(DATA_DIR, EDGES_PREFIX)
nodes_with_labels = nodes.merge(node_labels, on='address', how='left')

### Building the graphs

In [ ]:
graphs = build_emergence_graphs_for_time_range(
    edges_with_labels_df=edges_with_edge_labels,
    nodes_with_classes_df=nodes_with_labels,
    first_time_step=1,
    last_time_step=10,
    max_walk_length=2,
    time_horizon=3,
    use_distance_labels=True,
    keep_class_labels_as_features=False,
    ignore_illict=True,
    ignore_previously_transacting_with_illicit=True
)

Total unique addresses across all time: 822942
Total time steps: 49
Generating 10 graphs (time steps 1 to 10)...

t=1: nodes= 34853, edges=  66836, labels={np.int64(-1): 34853}
